In [269]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix




## Movielens full - EDA
a) Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan,
utan försök undersöka fler aspekter av datan.

* Jag börjar med att undersöka vad datasetet har för columner med df.head och df.info()
* Samt så har jag läst på "Read me" filen som tillhör datasetet

In [270]:
df_movies= pd.read_csv("Data/ml-latest/movies.csv")
df_movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [271]:
# Jag tar df.info 
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  86537 non-null  int64 
 1   title    86537 non-null  object
 2   genres   86537 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [272]:
df_ratings= pd.read_csv("Data/ml-latest/ratings.csv")
df_ratings.head()

MemoryError: Unable to allocate 774. MiB for an array with shape (3, 33832162) and data type int64

In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [ ]:
#MovieID to movie name mapping (Youtube Spencer Pao)
movie_names= df_movies.set_index('movieId')['title'].to_dict()
n_users = len(df_ratings.userId.unique())
n_items = len(df_ratings.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_users)
print("The ful rating matrix will have:", n_users*n_items,'elements.')
print('.............')
print("Number of ratings:",len(df_ratings))
print("Therefore:", len(df_ratings)/(n_users*n_items)*100, '% of the matrix is filled.') 


Number of unique users: 330975
Number of unique movies: 330975
The ful rating matrix will have: 27550028025 elements.
.............
Number of ratings: 33832162
Therefore: 0.12280264096028991 % of the matrix is filled.


In [ ]:
import torch
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm



class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # Skapa användarinbäddningar
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        # Skapa objektinbäddningar
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
    
    def forward(self, data):
        # Matrixmultiplikation
        users, items = data[:, 0], data[:, 1]
        user_embedding = self.user_factors(users)
        item_embedding = self.item_factors(items)
        return (user_embedding * item_embedding).sum(1)
    
    def predict(self, user, item):
        return self.forward(torch.tensor([[user, item]], dtype=torch.long))


In [ ]:
#Creating the data loader (nececery for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

class Loader(Dataset):
    def __init__(self):
        self.ratings = df_ratings.copy()
        
        #Extract all user IDs and movie IDs
        users = df_ratings.userId.unique()
        movies = df_ratings.movieId.unique()
        
        #Unique values: index
        self.userid2idx = {i:i for i,o in enumerate(users)}
        self.movieid2idx = {i:i for i,o in enumerate(movies)}
        
        #Obtiaind continuoues ID for users and movies
        self.idx2userid = {i:o for o, i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o, i in self.movieid2idx.items()}
        
        #return the id from the index values as noted in the lambada function down below.
        #self.ratings.movieId = df_ratings.movieId.apply(lambda x: self.movieid2idx[x])
        #self.ratings.userId =df_ratings.userId.apply(lambda x: self.userid2idx[x])
        self.ratings.movieId = df_ratings.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = df_ratings.userId.apply(lambda x: self.userid2idx[x])

        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y =self.ratings['rating'].values
        self.x , self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors(ready for torch models.)
        
    def __getitem__ (self, index):
        return (self.x[index], self.y[index])
        
    def __len__(self):
        return len(self.ratings)
        

In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        
#GPU enable if yu have a GPE...
if cuda:
    model = model.cuda()
# MSE loss
loss_fn = torch.nn.MSELoss()

#ADM optimizer
optimizer = torch.nn.MSELoss()

#Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle = True)
     

Is running on GPU False
MatrixFactorization(
  (user_factors): Embedding(330975, 8)
  (item_factors): Embedding(83239, 8)
)
user_factors.weight tensor([[0.0016, 0.0460, 0.0457,  ..., 0.0406, 0.0074, 0.0292],
        [0.0135, 0.0213, 0.0418,  ..., 0.0385, 0.0457, 0.0416],
        [0.0089, 0.0082, 0.0255,  ..., 0.0371, 0.0372, 0.0092],
        ...,
        [0.0175, 0.0378, 0.0175,  ..., 0.0077, 0.0498, 0.0301],
        [0.0426, 0.0036, 0.0253,  ..., 0.0057, 0.0006, 0.0094],
        [0.0100, 0.0151, 0.0010,  ..., 0.0019, 0.0050, 0.0342]])
item_factors.weight tensor([[0.0215, 0.0201, 0.0015,  ..., 0.0098, 0.0409, 0.0119],
        [0.0371, 0.0334, 0.0013,  ..., 0.0255, 0.0008, 0.0440],
        [0.0380, 0.0029, 0.0320,  ..., 0.0242, 0.0134, 0.0414],
        ...,
        [0.0406, 0.0037, 0.0476,  ..., 0.0482, 0.0224, 0.0002],
        [0.0181, 0.0142, 0.0331,  ..., 0.0076, 0.0321, 0.0292],
        [0.0060, 0.0089, 0.0210,  ..., 0.0450, 0.0394, 0.0208]])


Exception ignored in: <function tqdm.__del__ at 0x000002D6E9095EE0>
Traceback (most recent call last):
  File "c:\Users\saram\OneDrive\Dokument\GitHub\AI-Sara-Mellgren-OOPA23\venv\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\saram\OneDrive\Dokument\GitHub\AI-Sara-Mellgren-OOPA23\venv\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


MemoryError: Unable to allocate 258. MiB for an array with shape (33832162,) and data type int64

In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x,y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            loss = loss_fn(outputs.squeeeze(),y.type()(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses)/ len(losses))        

C:\Users\saram\AppData\Local\Temp\ipykernel_11756\1059283686.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Jag ser att jag behöver splita cellerna så att i genders och film titel delas upp.

In [ ]:
df_movies = df_movies.assign(year=df_movies ['title'].str.extract(r'\((\d{4})\)'), title=df_movies ['title'].str.replace(r'\(\d{4}\)', '').str.strip())

df_movies['genres'] = df_movies['genres'].str.split('|')
df_movies= df_movies.explode('genres')

df_movies.head()


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure,1995
0,1,Toy Story (1995),Animation,1995
0,1,Toy Story (1995),Children,1995
0,1,Toy Story (1995),Comedy,1995
0,1,Toy Story (1995),Fantasy,1995


b) Vilka är de 10 filmerna med flest ratings?

In [ ]:
# Utför sammanslagning samt kolumner samt återställt indexet till 'movieId'
#merged_df = pd.merge(df_ratings, df_movies, on='movieId')[['movieId', 'title', 'genres', 'rating', 'timestamp']]
#merged_df = merged_df.set_index('movieId').reset_index()

# Gruppera efter filmtitel och räkna antalet betyg för varje film
#top_movies = merged_df.groupby('title').size().reset_index(name='rating_count')

# Sortera resultaten i fallande ordning baserat på antalet betyg och välj de 10 första raderna
#top_10_movies = top_movies.sort_values(by='rating_count', ascending=False).head(10)

#print(top_10_movies)


c) Beräkna den genomsnittliga ratingen för dessa 10 filmerna med flest ratings.

In [ ]:
avg_rating_top_10 = top_10_movies.merge(merged_df, on='title').groupby('title')['rating'].mean()

print(avg_rating_top_10)

title
Fight Club (1999)                   4.236019
Forrest Gump (1994)                 4.068189
Inception (2010)                    4.176187
Jurassic Park (1993)                3.689013
Lion King, The (1994)               3.833477
Matrix, The (1999)                  4.160631
Pulp Fiction (1994)                 4.191778
Shrek (2001)                        3.748595
Silence of the Lambs, The (1991)    4.150287
Toy Story (1995)                    3.893508
Name: rating, dtype: float64


d) Gör en plot över årtal och antalet filmer representerade i datasetet.

Jag tar filtrerar ut så att ploten bara visar filmer från 1950 och framåt

In [ ]:
print(merged_df.head())


   movieId             title     genres  rating   timestamp
0        1  Toy Story (1995)  Adventure     4.0  1225734739
1        1  Toy Story (1995)  Animation     4.0  1225734739
2        1  Toy Story (1995)   Children     4.0  1225734739
3        1  Toy Story (1995)     Comedy     4.0  1225734739
4        1  Toy Story (1995)    Fantasy     4.0  1225734739


In [ ]:
print(merged_df['timestamp'].describe())

count    9.211335e+07
mean     1.277818e+09
std      2.529235e+08
min      7.896520e+08
25%      1.060876e+09
50%      1.284173e+09
75%      1.500351e+09
max      1.689843e+09
Name: timestamp, dtype: float64




e) Gör en plot över antalet ratings mot movieId.

f) Beräkna genomsnittliga ratings för de top 10 filmerna med flest ratings. Gör ett stapeldiagram över
dessa.

1.2 Skapa gles matris

Likt i videon i uppgift 1.0 skapade du en pivottabell av dataframet med index: "movieId", columns: "userId"
och values: "ratings". Denna pivottabell är dock "dyr" att skapa och förmodligen kommer inte din dator att
klara av skapa den om du inte filtrerar bort viss data. Fundera ut ett lämpligt sätt att filtrera ditt dataset,
pröva dig fram och motivera.#

Skapa en gles (sparse) matris av denna pivottabell mha scipy.sparse.csc_matrix(). Vill du använda
dig av all data går det också att lösa, men du behöver lösa hur du skapar den glesa matrisen utan pandas
pivot-tabell.

 1.3 Recommender system

Skapa ett recommender system med KNN och låt systemet ta input från användaren och skriva ut top 5
rekommenderade filmerna, baserat på användarens sökquery. Observera att det finns ett logiskt fel i
videon som gör att rekommendationerna inte blir så bra, försök hitta felet och åtgärda det.

a) Beskriv med ord hur ditt system fungerar.

b) Leta online och läs vidare om rekommenderarsystem och beskriv kort hur dem fungerar. Glöm inte
källhänvisa.

